# Kapitel 14 - Vertiefende Themen zur Textklassifikation

## 14.1. Kapitelübersicht <a class="anchor" id="14-1"/>

Sie sind nun an dem Punkt angekommen, an dem Sie kein Anfänger mehr in der Textklassifikation sind. Sie haben die wichtigsten Machine Learning Textklassifikationsverfahren kennen gelernt und einen Einblick in das Deep Learning erhalten. In diesem Kapitel werden weitere Themen vorgestellt, die für eine weiterführende Beschäftigung(?) mit der Textklassifikation spannende und wichtige Themen sind, den Rahmen dieser Reihe jedoch sprengen würden. Einige dieser Themen werden als Masterveranstaltungen für Digital Humanities oder der Informatik angeboten und behandeln fortgeschrittenere Themen. In diesem Kapitel möchte ich einen kurzen theoretischen Einblick in die verschiedenen Themen geben und erste Anlaufstellen für diese Themen liefern (AF!). Ab jetzt wird es jedoch kaum Codebeispiele mehr geben (AF!!).

<b>Abschnittsübersicht</b><br>

[14.1. Kapitelübersicht](#14-1)<br>
[14.2. Weitere Textrepräsentationsmöglichkeiten](#14-2)<br>
[14.2.1. N-Gramme](#14-2-1)<br>
[14.2.2. Word Embeddings](#14-2-2)<br>

Am Ende dieses Kapitel werden wir folgende Themen behandelt und/oder vertieft haben:
- N-Gramme
- Benutzung von N-Grammen mit Scikit learn Vectorizern
- Limitationen des Bag-of-Words-Modell
- Word Embeddings
- 

## 14.2. Weitere Textrepräsentationsmöglichkeiten <a class="anchor" id="14-2"/>

### 14.2.1. N-Gramme <a class="anchor" id="14-2-1"/>

Bis zu diesem Kapitel hatten wir immer eine einfache Bag-of-Words Repräsentation für unsere Daten benutzt. In Kapitel 4 hatten wir diese durch eine Tf-Idf-Gewichtung erweitert. Eine andere Erweiterung des Bag-of-Words Modells kann mit <b>N-Grammen</b> erreicht werden. Der Text wird dabei in $n$ aufeinanderfolgende Teile zerlegt. Ein Feature ist nun nicht mehr nur ein Wort oder Zeichen, sondern z.B. $n$-Wörter. So kann der Text "Die Hauptstadt ist Berlin" in die N-Gramme "Die Hauptstadt", "Hauptstadt ist", "ist Berlin" zerlegt werden, wenn $n\ =\ 2$.<br>

In Scikit learn können wir die N-Gramme als Parameter `ngram_range` unseres Vectorizers angeben. Dieser Parameter ist ein Tupel mit zwei Werten. Der erste Wert bezeichnet die untere Grenze des "Gramms" und der zweite Wert die obere Grenze. Bei `ngram_range=(1,2)` wird der Satz "Die Hauptstadt ist Berlin" in die N-Gramme "Die", "Die Hauptstadt", "Hauptstadt", "Hauptstadt ist", "ist", "ist Berlin", "Berlin" zerlegt werden. Umso größer man die N-Gramme setzt, umso länger dauert auch die Ausführung des Codes, da mehr Features berücksichtig werden müssen. Dadurch kann das Bag-of-Words-Modell jedoch auch mehr Bedeutungen aus den Texten extrahieren. Häufig kann dadurch die Genauigkeit der Klassifizierung verbessert werden, in unserem Fall ist sie jedoch etwas schlechter geworden (der F1-score ohne N-Gramm ist 0.87).

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
import numpy as np

import pandas as pd
corpus = pd.read_csv("tutorialdata/corpora/wikicorpus_v2.csv", index_col=0)


labels = LabelEncoder().fit_transform(corpus["category"])
vector  = TfidfVectorizer(ngram_range=(1,2)).fit_transform(corpus["text"])


X_train, X_test, y_train, y_test = train_test_split(vector, 
                                                    labels, 
                                                    test_size=0.2, 
                                                    train_size=0.8,
                                                    random_state=42)

  
# Multinomial Naive Bayes
mnb_classifier = MultinomialNB()
mnb = mnb_classifier.fit(X_train, y_train)

# cross validation des Trainingsdatensatzes
mnb_scores = cross_val_score(mnb_classifier, vector, labels, cv=3)
mnb_mean = np.mean(mnb_scores)

print("Der Mittelwert der cross validation bei der  Klassifizierung " 
      + f" mit Multinomial Naive Bayes ist {str(np.around(mnb_mean, decimals=3))}."
      + "\n")


# F1-score des Testdatensatzes
y_pred = mnb_classifier.predict(X_test)
mnb_f1 = f1_score(y_test, y_pred, average="micro")

print("Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist "
      + f"{str(np.around(mnb_f1, decimals=3))}.")

Der Mittelwert der cross validation bei der  Klassifizierung  mit Multinomial Naive Bayes ist 0.852.

Der F1-score für die Klassifizierung mit Multinomial Naive Bayes ist 0.86.


TODO lesen: https://www.analyticsvidhya.com/blog/2017/06/word-embeddings-count-word2veec/?utm_source=blog&utm_medium=top-pretrained-models-nlp-article → vllt. CBOW nutzen?

### 14.2.2. Word Embeddings <a class="anchor" id="14-2-2"/>

#### Grenzen des Bag-of-Words Modell

Das Bag-of-Words Modell ist sehr einfach zu verstehen und simpel zu implementieren. Es ist flexibel und kann z.B. mit der tf-idf-Gewichtung erweitert werden. Jedoch ist das Bag-of-Words Modell auch limitiert: Eines der größten Schwächen ist die Missachtung der semantischen Beziehung von Wörtern, Synonyme etwa werden ignoriert. Weiterhin hat das Bag-of-Words Modell das Problem, dass je größer das Vokabular wird, desto größer können einzelne Wortvektoren werden. Diese sind sparse, da sie fast nur aus Nullen bestehen. Sparse Repräsentationen sind sehr rechenintensiv und schwierig zu modellieren, da die Textklassifizierungsverfahren Schwierigkeiten haben, die wenige Informationen im großen Repräsentationsraum zu nutzen. ÜBERPRÜFEN TODO

#### Alternative: Word Embeddings

<b>Word Embedding</b> (deutsch: Worteinbettung) ist die Sammelbezeichnung für eine Reihe von Sprachmodellierungstechniken. Sie bieten eine andere Form der Wortrepräsentation als das Bag-of-Words-Modell, bei der Wörter mit einer ähnlichen Bedeutung ähnlich dargestellt werden, wobei der <b>Kontext</b> der Wörter berücksichtigt wird. Word Embeddings repräsentieren Wörter als Vektoren in einem multidimensionalen semantischen Raum. In diesem Raum werden Wörter, die ähnlich zueinander sind, näher beieinander platziert. Diesen Raum kann man sich etwa folgendermaßen vorstellen:<br><br>

<img src="https://miro.medium.com/max/1291/1*Fat62b1ZITOFMPXTcHNkLw.jpeg" alt="Word Embedding Space" width="600px;"/>

Word Embeddings wurde ab 2013 durch die Einführung des Algorithmus <b>word2vec</b> populär, in den folgenden Jahren folgten weitere Word Embedding Algorithmen wie <b>GloVe</b> oder <b>FastText</b>. Einen guten Überblick zu diesen Algorithmen bietet der folgende <a href="https://galaxydatatech.com/2018/12/16/word-embedding/">Blog-Eintrag</a>. Hier ein paar grundlegende Tipps:

- Das Trainieren von eigenen Word Embeddings kann sehr lange dauern und erzielt nur bei sehr großen Datensätzen gute Ergebnisse. Bei einer Beschäftigung mit Word Embeddings sollte auf jeden Fall ein Blick auf die sogenannten <u>pretrained embeddings</u> (deutsch: vortrainierte Embeddings) geworfen werden, die bequem heruntergeladen werden können und zudem umfassende Sprachinformationen einbringen können. Hier ist eine Tabelle[<sup>1</sup>](#fn1) über die vortrainierten Word Embeddings:


| Eigenschaften | word2vec | glove | fasttext | elmo  | bert |
|---|---|---|---|---|---|
|Entstehungsjahr | 2013  | 2014 | 2016 | 2018 | 2018 |
|Deutsche Modelle | x | x | x | x | x |
|<i>Out of vocabulary</i>| o | o | x | x | x |
|Kontextsensitiv| o | o | o | x | x |

- Eine sehr einfache und einsteigerfreundliche Benutzung von Word Embeddings bietet die NLP-Bibliothek <a href="https://github.com/zalandoresearch/flair"><b>Flair</b></a>. Flair stellt selbst sehr gute <a href="https://github.com/zalandoresearch/flair/tree/master/resources/docs">Tutorials</a> zur Benutzung ihrer Word Embeddings bereit.
- Word Embeddings sollten vorwiegend in Verbindung mit Deep Learning Modellen verwendet werden, die Benutzung mit Scikit learn ist nicht ganz unkompliziert und erfordert sehr viel Vorarbeit[<sup>2</sup>](#fn2). Eine Übersicht, wie man Word Embeddings mit Keras benutzt, bieten die Blog-Einträge: <a href="https://lekonard.github.io/blog/how-to-use-flair-with-keras/">How to use flair with keras</a> und <a href="https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html">Using pre-trained word embeddings in a Keras model</a> sowie Kapitel 6.1. von Chollets Deep Learning Buch.


<hr style="border: 0.1px solid black;"/>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">1</sup> &nbsp; Diese Tabelle wurde von Prof. Dr. Jannidis von der Julius-Maximilians-Universität erstellt.</span><br>
<span id="fn1" style="font-size:8pt; line-height:1"><sup style="font-size:5pt">2</sup> &nbsp; <a href="https://github.com/scikit-learn/scikit-learn/issues/8978">Hier</a> hat ein Github-User einen Transformer für Word Embeddings in Scikit learn erstellt, jedoch hat dieser keine besseren Werte erreicht.</span>

## 14.3. Weitere Arten von Neuronalen Netzen

### 14.3.1. Convolutional Neural Networks

TODO

### 14.3.2. Recurrent Neural Networks

TODO

## 14.4. Pretrained Language Models

- gute Übersicht: https://www.analyticsvidhya.com/blog/2019/03/pretrained-models-get-started-nlp/

Wie Boxplots interpretiert werden, kann <a href="https://de.khanacademy.org/math/statistics-probability/summarizing-quantitative-data/box-whisker-plots/a/box-plot-review">hier</a> nachgelesen werden.